## Single Party XGBoost on Data Subset
First we'll train an XGBoost model on a subset of the data. This simulates the federated setting in that a party will only have a subset of the data that's available to the central trusted server for training. We'll look at the performance of a XGBoost model that's only trained on this subset. 
![title](img/exercise1.png)

Import the necessary libraries

In [ ]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

Let `p` be your party ID. Load in and examine the training data partition belonging to your party, located at /data/training_data_<partition>.csv, to get a better understanding of the data. 

In [ ]:
# TODO: Read in the comma separated training data located at /data/<dataset>.csv using the pandas.read_csv() function
# and print out the first few rows of the dataset using the .head() function
# Make sure to specify header=None in the read_csv() call

training_data_subset = # TODO
training_data_subset.head()

In [ ]:
# Split the training dataset into features and labels
y_train_subset = training_data_subset.iloc[:, 0]
# TODO: examine the first 5 labels 

In [ ]:
x_train_subset = training_data_subset.iloc[:, 1:]
# TODO: examine the first 5 rows

Preprocess the test data located at /data/dataset_test.csv

In [ ]:
# TODO: split the test data into features and labels

Train the model with the training data. Feel free to play with the hyperparameters. For example, you may want to adjust the `n_estimators` hyperparameter, which adjusts the number of trees in the ensemble. For a full list of hyperparameters, go here: https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
model = xgb.XGBRegressor()
# TODO: fit the model to the training data

Get predictions and evaluate the model with the test data. Feel free to use different error functions. We suggest the sklearn mean_squared_error() or mean_absolute_error() functions.

In [1]:
# TODO: use the model to get predictions for the test set and calculate the prediction error

Compare your locally trained model's accuracy with those of other members of your federation. How do you think your locally trained model will perform on data that doesn't fit the distribution of your training data?